In [48]:
import tensorflow as tf

In [66]:
def input_fn(filenames, batch_size=2, num_epochs=1, perform_shuffle=False):
    print('Parsing', filenames)
    def decode_libsvm(line):
        columns = tf.string_split([line], ' ')
        labels = tf.string_to_number(columns.values[0], out_type=tf.float32)
        splits = tf.string_split(columns.values[1:], ':')
        id_vals = tf.reshape(splits.values,splits.dense_shape)
        feat_ids, feat_vals = tf.split(id_vals,num_or_size_splits=2,axis=1)
        feat_ids = tf.string_to_number(feat_ids, out_type=tf.int32)
        feat_vals = tf.string_to_number(feat_vals, out_type=tf.float32)
        return feat_ids, feat_vals, labels

    # Extract lines from input files using the Dataset API, can pass one filename or filename list
    dataset = tf.data.TextLineDataset(filenames).map(decode_libsvm, num_parallel_calls=10).prefetch(1000)

    # Randomizes input using a window of 256 elements (read into memory)
    if perform_shuffle:
        dataset = dataset.shuffle(buffer_size=256)

    # epochs from blending together.
    dataset = dataset.repeat(num_epochs)
    dataset = dataset.batch(batch_size) # Batch size to use

    iterator = dataset.make_one_shot_iterator()
    batch_feat_ids, batch_feat_vals,batch_labels = iterator.get_next()
    return batch_feat_ids, batch_feat_vals,batch_labels

In [51]:
filenames = './tr.mini.libsvm'

In [67]:
 , batch_feat_vals,batch_labels = input_fn(filenames)

Parsing ./tr.mini.libsvm


In [70]:
sess = tf.Session()
sess.run(tf.shape(batch_feat_ids))

array([ 2, 39,  1], dtype=int32)

In [71]:
sess.run(tf.shape(batch_feat_vals))

array([ 2, 39,  1], dtype=int32)

In [72]:
sess.run(tf.shape(batch_labels))

array([2], dtype=int32)

In [47]:
with tf.Session() as sess:
    Feat_Emb = tf.constant([[1.0,2.0,3.0],[4.0,5.0,6.0],[7.0,8.0,9.0],[10.0,11.0,12.0]])
    feat_ids = tf.constant([[[0],[2]],[[1],[3]]])
    print('feat_ids:'+ str(sess.run(feat_ids)))
    feat_ids = tf.reshape(feat_ids,shape=[-1,2])
    print('feat_ids:'+str(sess.run(feat_ids)))
    embeddings = tf.nn.embedding_lookup(Feat_Emb, feat_ids)
    print('embeddings: '+str(sess.run(embeddings)))
    feat_vals = tf.constant([[[0.1],[0.2]],[[0.5],[0.6]]])
    print('feat_vals: ' + str(sess.run(feat_vals)))
    feat_vals = tf.reshape(feat_vals,shape=[-1,2,1])
    xx = tf.multiply(embeddings,feat_vals)
    print('xx:' + str(sess.run(xx)))

feat_ids:[[[0]
  [2]]

 [[1]
  [3]]]
feat_ids:[[0 2]
 [1 3]]
embeddings: [[[ 1.  2.  3.]
  [ 7.  8.  9.]]

 [[ 4.  5.  6.]
  [10. 11. 12.]]]
feat_vals: [[[0.1]
  [0.2]]

 [[0.5]
  [0.6]]]
xx:[[[0.1       0.2       0.3      ]
  [1.4       1.6       1.8000001]]

 [[2.        2.5       3.       ]
  [6.        6.6000004 7.2000003]]]
